# Базовая линейная модель

В этом ноутбуке содержится код базовой линейной модели, которая показала наивысшее качество в результате ряда экспериментов, которые содержатся в отдельном ноутбуке Voronik_experiments.ipynb, а также проверочный код для инференса, кторый можно протестировать не ожидая обучения модели, так как к ноутбуку также приложен файл pipeline.pkl, который достаточно загрузить для выполнения инференса.

In [ ]:
import re
import string
import os
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Выполним загрузку ранее полученных файлов содержащих тренировочный и тестовый датасет. В тренировочном датасете содержатся 100 книг различных авторов, по одной книге, каждого автора (те по 1 объекту, каждого из 100 классов), а в тестовом датасете 51 книга части этих же авторов, но не та, которая попала в тренировочный набор. Обучение модели было выполнено на текстах книг, которые были очищены от метаданных и пробельных символов, а также обрезаны до максимальной длинны слов в 200000.

In [ ]:
df_train = pd.read_parquet('//content//drive//MyDrive//AI//NLP//df_train.pq')
df_test = pd.read_parquet('//content//drive//MyDrive//AI//NLP//df_test.pq')

## Описание признаков тренировочного и тестового датасетов
1.  'author' - имя и фамилия автора **(целевая переменная 100 классов)**
2.  'text_' - оригинальный полный текст книги без пробельных символов и метаданных
3.  'cnt_sent' - количество предложений
4.  'text_len' - количество символов в оригинальном тексте, до обрезки
5.  'text' - обрезанный в середине текст книги до максимальной длинны в 200000 слов
6.  'text_len2' - количество символов в "text"
7.  'words_cnt' - количество слов в обрезанном тексте
8.  'wrds_sent_cnt' - отношение количества слов к количеству предложений, или средняя длина предложения
9.  'cnt_words_unique' - количество уникальных слов
10. 'unwords_words' - отношение количества уникальных слов к количеству слов
11. 'median_word_length' - медианная длина слов
12. 'mean_word_length' - средняя длина слова
13. 'max_word_length' - максимальная длина слова
14. 'words_symbols' - отношение количества слов к количеству символов
15. 'words_dots' - отношение количества слов к количеству точек
16. 'words_commas' - отношение количества слов к количеству запятых
17. 'words_excls' - отношение количества слов к количеству восклицательных знаков
18. 'words_questions' - отношение количества слов к количеству вопросительных знаков
19. 'words_semicolons' - отношение количества слов к количеству точек с запятой
20. 'words_colons' - отношение количества слов к количеству двоеточий
21. 'words_dashs' - отношение количества слов к количеству тире
22. 'words_aposts' - отношение количества слов к количеству апострофов
23. 'words_ellipsis' - отношение количества слов к количеству многоточий
24. 'words_quots' - отношение количества слов к количеству кавычек
25. 'cnt_adv_freq' - словарь с количествами частотных наречий
26. 'cnt_swadesh_freq' - словарь с количествами слов из списка Сводеша
27. 'cnt_word_eng' - количество уникальных слов из словаря англ. языка
28. 'prc_wrds_not_eng' - отношение количества английских слов к количеству слов
29. 'uniq_word_cnt' - словарь с количествами уникальных слов
30. 'cnt_punct_frq' - словарь с количествами знаков пунктуации
31. 'lex_div' - (lexical_diversity) - лексическое разнообразие
32. 'tfidf_keywords' - ключевые слова с максимальными TF-IDF значениями
33. 'pos_frq' - словарь с количествами слов по частям речи
34. 'pos_cnt' - количество уникальных частей речи
35. 'ent_frq' - словарь с количествами слов именнованных сущностей
36. 'ent_cnt' - количество уникальных именнованных сущностей
37. 'uchars_frq' - словарь с количествами букв английского алфавита
38. 'uchars_cnt' - количество уникальных букв английского алфавита
39. 'fk_score' - показатель уровня читаемости текста по формуле Flesch-Kincaid

Оставим только данные необходимые для обучения модели

In [ ]:
ytrain = df_train['author']
ytest = df_test['author']

In [ ]:
Xtrain = df_train.text
Xtest = df_test.text
del df_train, df_test

Напишем функцию для подсчёта метрик классификации и составления отчёта

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

def calculate_multiclass_metrics(y_true, y_pred):
    """
    Рассчитывает основные метрики для многоклассовой классификации.

    :param y_true: Список или массив истинных значений классов.
    :param y_pred: Список или массив предсказанных значений классов.
    :return: Словарь с основными метриками.
    """

    metrics = {
        'accuracy': round(accuracy_score(y_true, y_pred), 4),
        'precision_macro': round(precision_score(y_true, y_pred, average='macro'), 4),
        'recall_macro': round(recall_score(y_true, y_pred, average='macro'), 4),
        'f1_macro': round(f1_score(y_true, y_pred, average='macro'), 4),
        'precision_micro': round(precision_score(y_true, y_pred, average='micro'), 4),
        'recall_micro': round(recall_score(y_true, y_pred, average='micro'), 4),
        'f1_micro': round(f1_score(y_true, y_pred, average='micro'), 4),
    }

    # Также можно вывести подробный отчет по каждому классу
    report = classification_report(y_true, y_pred)
    print("Classification Report:\n", report)

    return metrics

Наилучший результат в проведённых эксперементах показала модель логистической регрессии, с использованием OneVsRestClassifier на текстах обработанных векторизатором Tfidf с униграммами и биграммами и отнормированные MaxAbsScaler. Весь код обучения модели был собран в pipeline и затем сохранён при помощи joblib.

В результате данная модель показала такие метрики:<br>
 **accuracy**: 0.7255,<br>
 **precision_macro**: 0.6152,<br>
 **recall_macro**: 0.6727,<br>
 **f1_macro**: 0.6333,<br>
 **precision_micro**: 0.7255,<br>
 **recall_micro**: 0.7255,<br>
 **f1_micro**: 0.7255<br>

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
import joblib

vectorizer = TfidfVectorizer(ngram_range=(1, 2))
base_model = LogisticRegression(solver='liblinear')
ovr = OneVsRestClassifier(base_model)
scaler = MaxAbsScaler()
pipeline = make_pipeline(vectorizer, scaler, ovr)

pipeline.fit(train, ytrain)
lr_pred = pipeline.predict(test)

# joblib.dump(pipeline, 'pipeline.pkl')

lr_pred_prob = pipeline.predict_proba(test)
lr_probs = lr_pred_prob[:, 1]

calculate_multiclass_metrics(ytest.tolist(), lr_pred)

Classification Report:
                               precision    recall  f1-score   support

                       Aesop       0.50      1.00      0.67         1
             Agatha_Christie       1.00      1.00      1.00         1
             Alexandre_Dumas       1.00      1.00      1.00         1
             Alphonse_Daudet       1.00      1.00      1.00         1
               Anton_Chekhov       1.00      1.00      1.00         1
                Aristophanes       1.00      1.00      1.00         1
          Arthur_Conan_Doyle       1.00      1.00      1.00         1
             Charles_Dickens       0.00      0.00      0.00         1
             Dante_Alighieri       0.00      0.00      0.00         1
             Edgar_Allan_Poe       1.00      1.00      1.00         1
            Ernest_Hemingway       0.00      0.00      0.00         0
          F_Scott_Fitzgerald       1.00      1.00      1.00         1
                 Franz_Kafka       1.00      1.00      1.00      

{'accuracy': 0.7255,
 'precision_macro': 0.6152,
 'recall_macro': 0.6727,
 'f1_macro': 0.6333,
 'precision_micro': 0.7255,
 'recall_micro': 0.7255,
 'f1_micro': 0.7255}

Попробуем написать код для тестирования работы модели. В ячейке ниже при отправке модели текста о Кольце Всевластия из произведения Толкиена, модель выводит 3-х наиболее вероятных на её взгляд авторов, и Толкиен находится на втором месте немного уступив в вероятности авторства Сент-Экзюпери :)

При загрузке pipeline, и замене текста в коде ячейки можно проводить свои собственные эксперименты с моделью.

In [ ]:
import joblib

pipeline = joblib.load('pipeline.pkl')

def predict_top_3_authors(text):
    text_data = [text]
    predicted_probs = pipeline.predict_proba(text_data)
    top_3_indices = np.argsort(predicted_probs[0])[-3:][::-1]
    top_3_probs = predicted_probs[0][top_3_indices]
    top_3_authors = pipeline.named_steps['onevsrestclassifier'].classes_[top_3_indices]
    for author, prob in zip(top_3_authors, top_3_probs):
        print(f"Author: {author}, Probability: {prob}")

example_text ='''
One ring to rule them all, one ring to find them,
One ring to bring them all and in the darkness bind them.'''
predict_top_3_authors(example_text)

Author: Antoine_De_Saint_Exupery, Probability: 0.01396337039015265
Author: J_R_R_Tolkien, Probability: 0.013169534324720733
Author: Aesop, Probability: 0.012547013893284076
